In [1]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import joblib



In [2]:
# Step 1: Load the pretrained base model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Load the saved SVM model
svm_model = joblib.load('svm_model.pkl')

# Data generator for test images
datagen = ImageDataGenerator(rescale=1.0 / 255)
test_gen = datagen.flow_from_directory(
    r'S:\Datasets\Datasets\Animal Plant Dataset\New Plant Diseases Dataset(Augmented)\Test',
    target_size=(128, 128),
    batch_size=32,  # Reduce batch size
    class_mode='categorical',
    shuffle=False
)




Found 17572 images belonging to 38 classes.


In [3]:
# Extract features in batches
print("Extracting test features in batches...")
test_features = []
test_labels = []

for i in range(len(test_gen)):
    batch_data, batch_labels = test_gen[i]
    batch_features = base_model.predict(batch_data, verbose=1)
    test_features.append(batch_features)
    test_labels.append(batch_labels)
    if i >= len(test_gen) - 1:
        break

# Combine all batches
test_features = np.concatenate(test_features, axis=0)
test_labels = np.argmax(np.concatenate(test_labels, axis=0), axis=1) 

Extracting test features in batches...
1/1 [==============================] - 2s 2s/step


In [6]:
# Load the saved PCA model
pca = joblib.load('pca_model.pkl')
print("PCA model loaded successfully")

# Flatten and reduce dimensions using PCA
test_features_flattened = test_features.reshape(test_features.shape[0], -1)
test_features_reduced = pca.transform(test_features_flattened)

# Make predictions
print("Making predictions on test data...")
test_predictions = svm_model.predict(test_features_reduced)


PCA model loaded successfully
Making predictions on test data...


In [7]:
# Calculate accuracy
accuracy = accuracy_score(test_labels, test_predictions)
print(f"Accuracy on test data: {accuracy * 100:.2f}%")

Accuracy on test data: 94.26%


In [8]:
# Classification report
class_indices = {v: k for k, v in test_gen.class_indices.items()}
class_labels = [class_indices[i] for i in range(len(class_indices))]
print("\nClassification Report:")
print(classification_report(test_labels, test_predictions, target_names=class_labels))


Classification Report:
                                                    precision    recall  f1-score   support

Apple___Apple_scabbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb       0.90      0.92      0.91       504
Apple___Black_rotttttttttttttttttttttttttttttttttt       0.91      0.97      0.94       497
Apple___Cedar_apple_rusttttttttttttttttttttttttttt       0.96      0.96      0.96       440
Apple___healthyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy       0.92      0.94      0.93       502
Blueberry___healthyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy       0.94      0.96      0.95       454
Cherry_(including_sour)___Powdery_mildewwwwwwwwwww       0.95      1.00      0.97       421
Cherry_(including_sour)___healthyyyyyyyyyyyyyyyyyy       0.97      0.99      0.98       456
Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot       0.96      0.95      0.96       410
Corn_(maize)___Common_rust________________________       0.99      1.00      0.99       477
Corn_(maize)___Northern_Leaf_Blightttttttttttttttt     

In [ ]:
# Single image
from tensorflow.keras.preprocessing import image

def predict_single_image(img_path, model, base_model, pca, class_indices):
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Extract features using ResNet50
    features = base_model.predict(img_array)
    features_flattened = features.reshape(1, -1)  # Flatten
    features_reduced = pca.transform(features_flattened)  # PCA transformation

    # Predict the class
    prediction = model.predict(features_reduced)
    predicted_class = class_indices[np.argmax(prediction)]

    return predicted_class

# Usage example:
img_path = '/path/to/your/image.jpg'
predicted_label = predict_single_image(img_path, svm_model, base_model, pca, {v: k for k, v in test_gen.class_indices.items()})
print(f"Predicted class: {predicted_label}")
